Marcin Wardyński  
czwartek, 8:00

## Lab 9: Kontekstowe odpowiadanie na pytania

Do przygotowania danych treningowych i walidacyjnych dla modelu trzeba przerobić istniejące pliki json. Format nowych plików musi zawierać elementy:
- `id`
- `context`
- `question`
- `answers` z polem `text` zawierającym w postaci listy poprawne odpowiedzi

Poniższa funkcja dokonuje oczekiwanego przekształcenia na zbiorze PoQuAD: 

In [13]:
import json

def convert_data(data):
    results = []
    i = 0
    for article in data.get("data", []):
        for paragraph in article.get("paragraphs", []):
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                question = qa["question"]
                answers = qa['answers'] if 'answers' in qa.keys() else qa['plausible_answers']
                for answer in answers:
                    i += 1
                    results.append({
                        "id": i,
                        "context": context,
                        "question": question,
                        "answers": {
                            "text": [answer["generative_answer"]]
                        }
                    })
    return results


def convert_format(input_filepath, output_filepath):
    with open(input_filepath, "r", encoding="utf-8") as f:
        data = json.load(f)
    output_data = convert_data(data)
    output_wrapped_data = {"version": "0.1.0", "data": output_data}

    with open(output_filepath, "w", encoding="utf-8") as f:
        json.dump(output_wrapped_data, f, ensure_ascii=False, indent=2)


Wywołuję powyższą funkcję i zapisuję przekształcone dane w plikach `poquad-conv-train.json` oraz `poquad-conv-dev.json` dla zbiorów treningowego i walidacyjnego.

In [46]:
convert_format("poquad-train.json", "poquad-conv-train.json")
convert_format("poquad-dev.json", "poquad-conv-dev.json")

Do fine-tuningu swojego modelu użyłem nieznacznie zmienionego skryptu `run_seq2seq_qa.py` z repozytorium `transformers`, który został wywołany z następującymi parametrami:
```
  --model_name_or_path allegro/plt5-base \
  --train_file ../../../..poquad-conv-train.json \
  --validation_file ../../../..poquad-conv-dev.json \
  --context_column context \
  --question_column question \
  --answer_column answers \
  --do_train \
  --do_eval \
  --per_device_train_batch_size X \
  --learning_rate Ye-5 \
  --num_train_epochs 3 \
  --eval_strategy steps \
  --eval_steps 500 \
  --save_steps 500 \
  --predict_with_generate True \
  --metric_for_best_model f1 \
  --greater_is_better True \
  --load_best_model_at_end True \
  --save_total_limit 3 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ../../../../model_poquad_t5_base_f1_3e-5_b12
```

Jak widzimy powyżej, jako modelu pretrenowanego używam `allegro/plt5-base`, gdyż moja konfiguracja komputera nie pozwala mi na wykorzystanie pojemniejszego modelu. Do treningu przekazuję uprzednio przygotowane pliki ze zbiorem danych PoQuAD oraz wskazuję nazwy odpowiednich kolumn. Zlecam wykonanie treningu i ewaluacji z wielkością batcha i krokiem treningu zdefiniowanym dla każdego uruchomienia skryptu z innymi wartościami. Trening ma trwać zalecane trzy epoki, a wybrana strategia ewaluacji `f1` powinna zostać uruchomiona co 500 kroków. Finalnie powinien zostać zaladowany model o najlepszym wyniku ewaluacji. Parametry `max_seq_length` oraz `doc_stride` otrzymały wartości odpowiednio 384 i 128.

Wyniki skyptu dla wybranych parametrów treningowych:

| lr       | #batchy | eval em   | eval f1   |
|----------|---------|-----------|-----------|
| 2e-5     | 8       | 51.53     | 67.62     |
| 3e-5     | 12      | 50.84     | 66.87     |
| 4e-5     | 16      | 51.39     | 67.25     |
| **5e-5** | **16**  | **52.35** | **68.30** |

Jak widzimy, dla danych walidacyjnych zbioru PoQuAD najlepiej wypada model o stałej uczącej `5e-5` i rozmiarze batch-a `16` i to zarówno dla metryki *Exact Match*, jak i *f1*.

Skoro mamy już wytrenowany model wraz z jego wstępną ewaluacją, skupmy się na danych testowych. Poniższy zestaw funkcji buduje struktury słownikowe, które ułatwiają nawigowanie po danych testowych zawierających wskazane pytania prawne:

In [11]:
import json

NO_ANS = "no_ans"

class QA:
    def __init__(self, question_id, question, answer):
        self.question_id = question_id
        self.question = question
        self.answer = answer

class Entry:
    def __init__(self, passage_id, passage_text, qas):
        self.passage_id = passage_id
        self.passage_text = passage_text
        self.qas = qas

def init_qas_with_answers(filepath):
    qa_dict = {}
    with open(filepath, "r") as file:
        for line in file:
            record = json.loads(line.strip())
            
            if "score" in record and record["score"] == "1"\
                    and "question-id" in record and "answer" in record:
                qa = QA(record["question-id"], None, record["answer"])
                qa_dict[record["question-id"]] = qa
    return qa_dict

def match_questions_to_answers(filepath, qa_dict):
    q_wo_a = []
    with open(filepath, "r") as file:
        for line in file:
            record = json.loads(line.strip())
            
            if "text" in record and "_id" in record:
                if record["_id"] in qa_dict.keys():
                    qa = qa_dict[record["_id"]]
                    qa.question = record["text"]
                else:
                    qa_dict[NO_ANS].append(record["text"])


def organize_question_to_context_relations(filepath, qa_dict):
    qc_dict = {}
    with open(filepath, "r") as file:
        for line in file:
            record = json.loads(line.strip())
            
            if "score" in record and record["score"] == "1"\
                    and "passage-id" in record\
                    and "question-id" in record and record["question-id"] in qa_dict.keys():
                if record["passage-id"] not in qc_dict.keys():
                    qc_dict[record["passage-id"]] = []
                qc_dict[record["passage-id"]].append(record["question-id"])
    return qc_dict

def load_passages(filepath, qc_dict, qa_dict):
    entries = []
    with open(filepath, "r") as file:
        for line in file:
            record = json.loads(line.strip())
            
            if "text" in record and "_id" in record and record["_id"] in qc_dict.keys():
                qa_ids = qc_dict[record["_id"]]
                qas = []
                for qa_id in qa_ids:
                    qas.append(qa_dict[qa_id])
                entries.append(Entry(record["_id"], record["text"], qas))
    return entries

Po uruchomieniu tych fynkcji i przekazaniu odpowiednich plików wejściowych otrzymujemy następujące struktury:  
`qa_dict` - słownik łączący pytania z odpowiedziami  
`qc_dict` - słownik łączący pytania z kontekstem  
`test_passages` - lista z kontekstem oraz połączonymi z nim pytaniami i odpowiedziami  

Tak przygotowanych struktur będą wykorzystywać, aby zewaluować jakość modelu dla danych testowych.

In [2]:
qa_dict = init_qas_with_answers("simple-legal-questions-pl-main/answers.jl")

qa_dict[NO_ANS] = []
match_questions_to_answers("simple-legal-questions-pl-main/questions.jl", qa_dict)

qc_dict = organize_question_to_context_relations("simple-legal-questions-pl-main/relevant.jl", qa_dict)
test_passages = load_passages("simple-legal-questions-pl-main/passages.jl", qc_dict, qa_dict)

Dodatkowo napisałem funkcję przekształcającą dane zbioru PoQuAD do reprezentacji odpowiadającej danym testowym. Napisałem ją, gdyż chciałbym dodatkowo policzyć metryki dla zbioru walidacyjnego dokładnie tymi samymi funkcjami, co dla zbioru testowego.

In [35]:
def convert_poquad_data(filepath):
    val_entries = []
    with open(filepath, "r") as file:
        json_content = json.load(file)
        for data in json_content['data']:
            for paragraph in data['paragraphs']:
                qa_list = []
                for qa in paragraph['qas']:
                    answers = qa['answers'] if 'answers' in qa.keys() else qa['plausible_answers']
                    for answer in answers:
                        qa_obj = QA(None, qa['question'], answer['generative_answer'])
                        qa_list.append(qa_obj)
                    
                entry = Entry(None, paragraph['context'], qa_list)
                val_entries.append(entry)
    return val_entries

val_passages = convert_poquad_data("poquad-dev.json")

Funkcja `exec_passages` przechodzi po kontekstach w podanym zbiorze i przekazuje je do modelu wraz z pytaniami i odpowiedziami na nie. Wyjściowo otrzymujemy listę odpowiedzi modelu oraz oczekiwanych odpowiedzi generatywnych.

In [4]:
from tqdm import tqdm

def exec_passages(model, tokenizer, passages):
    answers = []
    expected_answers = []
    for passage in tqdm(passages):
        for qa in passage.qas:
            input_text = f"question: {qa.question} context: {passage.passage_text}"
            inputs = tokenizer(input_text, return_tensors="pt")

            outputs = model.generate(inputs["input_ids"], max_length=100, num_beams=5, early_stopping=True)

            answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
            answers.append(answer)
            expected_answers.append(qa.answer)
    return answers, expected_answers


Teraz wystarczy wczytać odpowiedni model i wyznaczyć jego odpowiedzi na pytania ze zbioru poprzez wywołanie powyższej funkcji. W pierwszej kolejności wykonuję te działania dla każdego z wytrenowanych modeli na zbiorze walidacyjnym następującymi wywołaniami:

In [42]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

lr5e_5_b16_t5_base_model_name = "./model_poquad_t5_base_f1_5e-5_b16"
lr5e_5_b16_t5_base_model_tokenizer = AutoTokenizer.from_pretrained(lr5e_5_b16_t5_base_model_name)
lr5e_5_b16_t5_base_model = AutoModelForSeq2SeqLM.from_pretrained(lr5e_5_b16_t5_base_model_name)

In [43]:
lr5e_5_b16_t5_base_val_answers, expected_val_answers = exec_passages(lr5e_5_b16_t5_base_model, lr5e_5_b16_t5_base_model_tokenizer, val_passages)

100%|██████████| 1453/1453 [2:08:29<00:00,  5.31s/it] 


Przedstawiam przykład tylko dla jednego z modeli, reszta została użyta w analogiczny sposób.

Następnie definiuję funkcje do obliczania miar *Exact Match* oraz *f1*.

Funkcja obliczająca *Exact Match* zawiera mały etap wstępnego przetworzenia tekstów na wejściu, który to usuwa skrajne spacje, znaki interpunkcyjne i zmniejsza czcionkę wszystkich słów, tak żeby lepiej odkrywać dokładne dopasowania.

In [ ]:
import string

def clean_text(s):
    s = s.strip()
    s = s.lower()
    s = ''.join(c for c in s if c not in string.punctuation)
    return s


def calculate_exact_matches(answers, expected_answers, clean_fun):
    matches = 0
    for s1, s2 in zip(answers, expected_answers):
        if clean_fun(s1) == clean_fun(s2):
            matches += 1
    return matches/len(answers)

I ostatecznie funkcja mierząca wartości TP, FP, FN i wyznaczająca na ich podstawie metryki *precision* i *recall* aby zwrócić opierającą się o nie metrykę *f1*. Poza dwoma odpowiedziami funkcja przyjmuje funkcję tokenizującą, która przeprowadza czyszczenie tekstu (takie samo, jak w przypadku *Exact Match*) i dodatkowo tworzy tokeny 1:1 ze słów w zdaniu.

In [79]:
from collections import Counter
import re

def tokenize(text):
    text = clean_text(text)
    tokens = re.split(r"[^\w]+", text)
    return tokens

def compute_single_f1(tokens1, tokens2):
    TP = sum((tokens1 & tokens2).values())
    FP = sum((tokens1 - tokens2).values())
    FN = sum((tokens2 - tokens1).values())

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1

def compute_f1(answers, expected_answers, tokenize_fun):

    f1_scores = []
    for s1, s2 in zip(answers, expected_answers):
        s1_t = Counter(tokenize_fun(s1))
        s2_t = Counter(tokenize_fun(s2))
        f1 = compute_single_f1(s1_t, s2_t)
        f1_scores.append(f1)

    return sum(f1_scores)/len(f1_scores)

Następnie wywołuję obydwie funkcje mierzące miary *Exact Match* i *f1* na zbiorze walidacyjnym.  
Powtarzam pomiary tych metryk, chociaż zostały mi już one zwrócone podczas treningu, gdyż chcę wykorzystać te same funkcje, których będą wykorzystywal później dla zbioru testowego. Chcę, żeby porównanie wyników dla zbioru walidacyjnego było bardziej spójne.

Jak widzimy powyżej, moje implementacje funkcji obliczających te metryki dodatkowo czyszczą wprowadzony tekst, a miara *f1* jest obliczana w sposób mikro (czyli liczymy *f1* dla każdego przypadku z osobna i uśredniamy już wyliczony zbiór wartości *f1*, żeby uzyskać wartość średnią dla zbioru).  
Co do funkcji użytych w skrypcie `run_seq2seq_qa.py`, to nie mam pewności w jaki sposób miary te zostały obliczone.

In [80]:
calculate_exact_matches(lr5e_5_b16_t5_base_val_answers, expected_val_answers, clean_text)

0.553399433427762

In [81]:
compute_f1(lr5e_5_b16_t5_base_val_answers, expected_val_answers, tokenize)

0.7320584815022007

Wyniki dla wywołań moich funkcji ewaluacyjnych na zbiorze walidacyjnym i dla wszystkich wytrenowanych modeli zamieszczam poniżej:

| lr       | #batchy | eval em   | eval f1   |
|----------|---------|-----------|-----------|
| 2e-5     | 8       | 0.544     | 0.723     |
| 3e-5     | 12      | 0.533     | 0.713     |
| 4e-5     | 16      | 0.538     | 0.718     |
| **5e-5** | **16**  | **0.553** | **0.732** |

Widzimy, że faktycznie wyliczone wartości różnią się od tych zwróconych przez skypt treningowy i są odrobinę wyższe. Ma to najpewniej swoje źródło w dodatkowym, minimalnym przygotowaniu danych do porównania, które stosuję.

Wciąż najlepszym modelem jest ten o rozmiarze batch-a 16 i stałej uczącej 5e-5.

Wiedząc, który model daje najlepsze wyniki dla zbioru walidacyjnego, odpowiedzmy przy jego użyciu na pytania ze zbioru testowego i sprawdźmy rezultaty:

In [77]:
lr5e_5_b16_t5_base_answers, expected_answers = exec_passages(lr5e_5_b16_t5_base_model, lr5e_5_b16_t5_base_model_tokenizer, test_passages)

100%|██████████| 557/557 [14:00<00:00,  1.51s/it]


In [82]:
calculate_exact_matches(lr5e_5_b16_t5_base_answers, expected_answers, clean_text)

0.29442508710801396

In [66]:
compute_f1(lr5e_5_b16_t5_base_answers, expected_answers, tokenize)

0.5210907914379148

Otrzymaliśmy następujące wyniki:

| lr   | #batchy | eval em | eval f1 |
|------|---------|---------|---------|
| 5e-5 | 16      | 0.294   | 0.521   |

Wartości metryk okazały się znacznie słabsze, niż dla zbioru walidacyjnego. Przyjrzyjmy się dziesięcu przykładom bliżej w poszukiwaniu przyczyny gorszych wyników.

In [85]:
import random

random.seed(7)
random_passages = random.sample(test_passages, 10)

random_passages_answers, expected_random_passages_answers = exec_passages(lr5e_5_b16_t5_base_model, lr5e_5_b16_t5_base_model_tokenizer, random_passages)

100%|██████████| 10/10 [00:22<00:00,  2.25s/it]


Ponieważ każdy kontekst zawiera tylko jedno pytanie, wyniki dla wylosowanych dziesięciu z nich możemy połączyć z wygenerowanymi odpowiedziami za pomocą liczby porządkowej w następujący sposób:

In [88]:
for i in range(len(random_passages)):
    print(f"kontekst: {random_passages[i].passage_text}")
    print(f"Pytanie: {random_passages[i].qas[0].question}")
    print(f"Oczekiwana odpowiedź: {random_passages[i].qas[0].answer}")
    print(f"Wygenerowana odpowiedź: {random_passages_answers[i]}")
    print()

kontekst: Art. 20. Rada Ministrów, na wniosek Ministra Skarbu Państwa, określa, w drodze rozporządzenia: 1) zakres, szczegółowe zasady i tryb kontroli, o której mowa w art. 2 pkt 7 i 8, 2) tryb powierzania mienia kierownikom urzędów państwowych, 3) tryb powierzania mienia kierownikom jednostek organizacyjnych podporządkowanych kierownikom urzędów państwowych oraz tryb udzielania im pełnomocnictw do reprezentowania Skarbu Państwa, 4) szczegółowe zasady ewidencjonowania majątku Skarbu Państwa, w tym zbiorczej ewidencji, o której mowa w art. 2 pkt 3, oraz związane z tym obowiązki państwowych jednostek organizacyjnych, którym powierzono to mienie.
Pytanie: Na czyj wniosek rada ministrów określa tryb powierzania mienia kierownikom urzędów państwowych?
Oczekiwana odpowiedź: na wniosek Ministra Skarbu Państwa
Wygenerowana odpowiedź: Ministra Skarbu Państwa

kontekst: Art. 64. 1. Kontrolę gospodarki finansowej powiatu sprawuje regionalna izba obrachunkowa. 2. Z zastrzeżeniem przepisów tego roz

In [89]:
calculate_exact_matches(random_passages_answers, expected_random_passages_answers, clean_text)

0.4

In [90]:
compute_f1(random_passages_answers, expected_random_passages_answers, tokenize)

0.6053930461073318

Ręczna ewaluacja odpowiedzi:

| Art nr | zgodność odpowiedzi          | uwagi                                                                                                           |
|--------|------------------------------|-----------------------------------------------------------------------------------------------------------------|
| 20     | zachowana                    | minimalistyczna odpowiedź modelu                                                                                |
| 64     | niezachowana (lecz poprawna) | odpowiedź oczekiwana nie zawiera się w kontekście nasz model odpowiada zgodnie z tekstem dostarczonego przepisu |
| 35     | zachowana                    | dokładne dopasowanie                                                                                            |
| 31     | niepełna                     | brak informacji o dacie i miejscu wystawienia oraz podpisie pracodawcy                                          |
| 36     | niezachowana (lecz poprawna) | odpowiedź oczekiwana nie zawiera się w kontekście nasz model odpowiada zgodnie z tekstem dostarczonego przepisu |
| 4      | niezachowana                 | pytanie trudne do odpowiedzi i dla człowieka - brak konkretnych dany                                            |
| 90     | zachowana                    | dokładne dopasowanie                                                                                            |
| 111    | zachowana                    | minimalistyczna odpowiedź modelu                                                                                |
| 159    | zachowana                    | dokładne dopasowanie                                                                                            |
| 29     | zachowana                    | dokładne dopasowanie                                                                                            |

Wyniki zinterpretuję odpowiadając na pytanie otwarte nr 2.

### Pytania otwarte

#### Czy jakość na zbiorze testowym odpowiada tej dla zbiru walidacyjnego?
Niestety nie, metryki *Exact Match* oraz *f1* wypadają znacznie gorzej dla zbioru testowego, niż dla zbioru walidacyjnego. Interpretacja takiego wyniku znajduje się w kolejnym pytaniu, gdzie przyglądam się bliżej małemu wycinkowi przeprowadzonego testu bliżej.

#### Jakie wyniki dostarcza model dla pytań testowych? Czy są one satysfakcjonujące?
Akurat wyniki dla wylosowanego podzbioru danych testowych wypadły nieco lepiej, niż ocena całego zbioru. Otrzymały one ocenę *Exact Match = 0.4* i *f1 = 0.605* co jest lepszym, od tego dla całego zbioru: *Exact Match = 0.294*, *f1 = 0.521*. Pomijając aspekt szczęśliwego losowania i analizując wykonane zapytania, można dojść do następujących wniosków:
- model odpowiada w sposób minimalistyczny, a oczekiwana odpowiedź jest często sformułowana pełnym zdaniem, stąd metryki oceniające zostają zaniżone. W wylosowanym podzbiorze danych testowych, zachowanie to dotyczy 2/10 przypadków.
- zbiór danych zawiera oczekiwane odpowiedzi, których ustalenie nie jest możliwe na podstawie dostarczonego kontekstu, stąd model odpowiadający zgodnie z informacją kontekstową oceniony jest negatywnie. Pytania takie powinny zostać oznaczone w zbiorze jako niemożliwe do odpowiedzi, co nie jest wykonane. Zachowanie dotyczy 2/10 przeanalizowanych przypadków.
- model potrafi zwrócić niepełną odpowiedź. Przypadek dotyczy 1/10 przypadków
- lub wręcz niepoprawną odpowiedź - 1/10 przypadków.

Biorąc pod uwagę powyższe uwagi możemy podsumować, że wśród przeanalizownych przypadków 4/10 to dokładne dopasowania, 2/10 są w pełni poprawne, lecz sformułowane lakonicznie, 2/10 niezgodne z oczekiwaną odpowiedzią, lecz w pełni zgodne z kontekstem i poprawne, 1/10 częściowo poprawne i 1/10 niepoprawne.

Podsumowując liczby z akapitu powyżej, mamy osiem poprawnych odpowiedzi, jedną częściowo poprawną i jedną niepoprawną, czyli w ponad 80% można być zadowolonym z pracy modelu, co jest wynikiem mnie satysfakcjonującym.  
Kiepska ocena modelu uwzględnionymi metrykami ma swoje źródło w danych testowych, które nie są konsekwentne co do odpowiadania równoważnikami zdań, czy też pełnymi zdaniami, oraz zawierającymi odpowiedzi spoza wiedzy zawartej w kontekście bez oznaczenia takich odpowiedzi odpowiednią adnotacją.

Co ciekawe, oczekiwałem większego wpływu fleksji języka polskiego na zaniżenie ocen sprawdzonych metryk, ale przypadki wylosowane na to nie wskazują. Przyjrzę się temu zagadnieniu bliżej w ramach odpowiedzi na kolejne pytanie, które dotyka sprawy fleksji języka.

#### Dlaczego ekstraktywne odpowiadanie na pytania nie jest właściwe dla językow z bogatą fleksją?
W ćwiczeniu używamy abstraktywnego QA, które zaskakująco dobrze radzi sobie z fleksją języka. Postanowiłem przeprowadzić dodatkowy eksperyment i uzupełnić wyliczanie metryki *f1* o dodatkową lematyzację słów pakietem `SpaCy`. W wynikach zawartych poniżej widać, że lematyzacja podniosła wartość *f1* zaledwie do 0.535, czyli o ok 1.5 punktu procentowego względem początkowego wyniku. Pokazuje to jasno, że fleksja nie jest problemem niskich wartości metryk, a ich przyczyna została opisana w odpowiedzi na poprzednie pytanie.

Przejdźmy jednak do odpowiedzi na postawione pytanie, które dotyczy modeli QA ekstraktywnych, a nie przebadanego abstraktywnego.  
Języki z bogatą fleksją już na poziomie przygotowywania modelu wpływają na jego działanie. Mianowicie model ten musi uwzględnić fleksję zarówno przy tokenizacji, jak i przy budowaniu osadzeń dla wczytanych tokenów. Fleksja prowadzi najpewniej do zwiększenia liczby unikalnych tokenów obsługiwanych przez model, oraz musi zostać uwzględniona w budowie kontekstu dla poszczególnych tokenów, przez co potrzebujemy pojemniejszego modelu i większej liczby zdań w korpusie uczącym, aby dobrze odwzorować zachodzące zależności fleksyjne <- właściwie ta trudność dotyczy zarówno modelu ekstraktywnego, jak i abstraktywnego QA.

Dodatkowo, model ekstraktywnego QA cechuje się mniejszą elastycznością, gdyż musi wskazać dokładny początek i koniec odpowiedzi w dostarczonym tekście, natomiast języki bogate fleksyjnie z mojego doświadczenia dają większą swobodę co do budowy zdań i kolejności zawartych w nim części zdania - przykładowo język angielski ma ściślej określoną strukturę zdania, niż np. polski, gdyż jego fleksja jest bardzo uboga, a gdzieś informacja o relacji pomiędzy wyrazami w zdaniu musi zostać zawarta. Skoro zdania języków bogatych fleksyjnie mogą być bardziej zróżnicowane, model mający za zadanie znalezie w tekście początku i końca odpowiedzi ma utrudnione zadanie.

#### Dodatkowe wyniki uzyskane przy realizacji zadania

Poniżej dodatkowy eksperyment z lematyzacją słów przed zastosowniem metryki *f1*:

In [27]:
!python -m spacy download pl_core_news_lg

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.7/573.7 MB 3.6 MB/s eta 0:00:0000:0100:05
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_lg')


In [ ]:
import spacy

nlp = spacy.load("pl_core_news_lg")

def tokenize_and_lemmatize(text):
    tokens = tokenize(text)
    text = " ".join(tokens)

    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    return lemmatized_tokens

In [91]:
compute_f1(lr5e_5_b16_t5_base_answers, expected_answers, tokenize_and_lemmatize)

0.5351370990492306

Obliczyłem z ciekawości też wydajności pozostałych modeli na danych testowych. Okazało się, że model o najmniejszej stałej uczącej daje najlepsze wyniki, lecz skoro nie był najlepszy dla danych walidacyjnych, to szanując zasady przeprowadzonych badań nie użyłem go w zasadniczej części zbierania wynikow.

| lr   | #batchy | eval em | eval f1 |
|------|---------|---------|---------|
| 2e-5 | 8       | 0.305   | 0.535   |
| 3e-5 | 12      | 0.287   | 0.5257  |
| 4e-5 | 16      | 0.293   | 0.5260  |

Na końcu mała funkcja pomocnicza, która zapisuje i wczytuje obliczone odpowiedzi. Szczególnie istotna, gdyż wyznaczanie odpowiedzi przez model w sposób czysto sekwencyjny zajmowało ok 2h dla danych walidacyjnych i ok 0.5h dla zbioru testowego

In [33]:
import json

def store_answers_in_json(answers, file_path):
    with open(file_path, "w") as f:
        json.dump(answers, f)

def load_answers_from_json(file_path):
    with open(file_path, "r") as f:
        return json.load(f)

In [76]:
store_answers_in_json(lr5e_5_b16_t5_base_val_answers, "lr5e_5_b16_t5_base_val_answers.json")